In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import nltk
from nltk.metrics.distance import edit_distance

# Load the pre-trained sentiment analysis model and tokenizer
MODEL_NAME = "lxyuan/distilbert-base-multilingual-cased-sentiments-student"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# Function to get sentiment scores for a given sentence
def get_sentiment_scores(sentence):
    """
    Computes sentiment scores for a given sentence using the pre-trained model.
    Returns a dictionary with positive, neutral, and negative scores.
    """
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy()[0]
    return {
        "positive": scores[0],
        "neutral": scores[1],
        "negative": scores[2]
    }

# Function to find a matching sentence with similar sentiment scores
def find_matching_sentence(target_sentence, decimal_places=5):
    """
    Compares two user-provided sentences: a target sentence and a candidate sentence.
    Ensures that the candidate sentence has sentiment scores matching the target sentence
    when rounded to the specified decimal places.
    Additionally, enforces a minimum Levenshtein distance of 30 and requires the candidate
    sentence to be between 40 and 60 characters in length.
    If no match is found, the function will return the target sentence scores.
    """
    target_scores = get_sentiment_scores(target_sentence)
    rounded_target_scores = {k: round(v, decimal_places) for k, v in target_scores.items()}

    found_sentence = None
    matched_scores = None
    for candidate_sentence in get_user_provided_sentence():

        # Ensure sentence length constraint (40-60 characters)
        if not (40 <= len(candidate_sentence) <= 60):
            continue

        # Ensure minimum Levenshtein distance of 30
        if edit_distance(target_sentence, candidate_sentence) < 30:
            continue

        # Compute sentiment scores for the candidate sentence
        candidate_scores = get_sentiment_scores(candidate_sentence)
        rounded_candidate_scores = {k: round(v, decimal_places) for k, v in candidate_scores.items()}

        # Check if the rounded sentiment scores match
        if rounded_candidate_scores == rounded_target_scores:
            found_sentence = candidate_sentence
            matched_scores = candidate_scores
            break

    return found_sentence, target_scores, matched_scores

# Function to get a user-provided candidate sentence
def get_user_provided_sentence():
    """
    Asks the user to input a sentence for comparison.
    """
    return [input("Enter a candidate sentence: ")]

# Example usage
target_sentence = input("Enter the target sentence: ")
matching_sentence, target_scores, matched_scores = find_matching_sentence(target_sentence, decimal_places=5)

# Print results in a single-line table format
print(f"| {'Sentence Type':<20} | {'Sentence':<50} | {'Positive':<10} | {'Neutral':<10} | {'Negative':<10} |")
print(f"| {'Target Sentence':<20} | {target_sentence:<50} | {target_scores['positive']:.5f} | {target_scores['neutral']:.5f} | {target_scores['negative']:.5f} |")
if matching_sentence:
    print(f"| {'Matching Sentence':<20} | {matching_sentence:<50} | {matched_scores['positive']:.5f} | {matched_scores['neutral']:.5f} | {matched_scores['negative']:.5f} |")
else:
    print("No matching sentence found")

Enter the target sentence: vv
Enter a candidate sentence: vv
| Sentence Type        | Sentence                                           | Positive   | Neutral    | Negative   |
| Target Sentence      | vv                                                 | 0.43669 | 0.26179 | 0.30153 |
No matching sentence found
